source : https://nominatim.org/release-docs/latest/api/Search/

https://medium.com/@adri.espejo/getting-started-with-openstreetmap-nominatim-api-e0da5a95fc8a

In [1]:
import pandas as pd
import numpy as np
import requests

hdb_string = './resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv'

C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv(hdb_string)

In [3]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113959 entries, 0 to 113958
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                113959 non-null  object 
 1   town                 113959 non-null  object 
 2   flat_type            113959 non-null  object 
 3   block                113959 non-null  object 
 4   street_name          113959 non-null  object 
 5   storey_range         113959 non-null  object 
 6   floor_area_sqm       113959 non-null  float64
 7   flat_model           113959 non-null  object 
 8   lease_commence_date  113959 non-null  int64  
 9   remaining_lease      113959 non-null  object 
 10  resale_price         113959 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 9.6+ MB


In [5]:
df['Address'] = df['block'] + " " + df['street_name']
# Prints only every 20 entries, This reduces total dataset of 113959 to a much manageable number
# API only accepts certain format so having them fed into the API as 'Block number' 'Street Name ' works best
addresslist = list(df['Address'])

In [6]:
addresslist[:10]

['406 ANG MO KIO AVE 10',
 '108 ANG MO KIO AVE 4',
 '602 ANG MO KIO AVE 5',
 '465 ANG MO KIO AVE 10',
 '601 ANG MO KIO AVE 5',
 '150 ANG MO KIO AVE 5',
 '447 ANG MO KIO AVE 10',
 '218 ANG MO KIO AVE 1',
 '447 ANG MO KIO AVE 10',
 '571 ANG MO KIO AVE 3']

In [10]:
all_coordinates = []

for index, address in enumerate(addresslist):
    search_query = 'https://nominatim.openstreetmap.org/search?q=' + address + '&countrycodes=sg&limit=1&format=json'
    print('\n','*'*25)
    print('Searching for address ' + address + '\n')
    print('Progress {} percent \n'.format(np.round(index/len(addresslist),2)))
    res = requests.get(search_query)
    search_result = eval(res.text)
    all_coordinates.append([search_result[0]['lat'], search_result[0]['lon']])
    print('Information Found and appended \n')
    print('*'*25 , '\n')


 *************************
Searching for address 406 ANG MO KIO AVE 10

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 108 ANG MO KIO AVE 4

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 602 ANG MO KIO AVE 5

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 465 ANG MO KIO AVE 10

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 601 ANG MO KIO AVE 5

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 150 ANG MO KIO AVE 5

Progress 0.0 percent 

Information Found and appended 

************************* 


 *************************
Searching for address 447 ANG MO K

IndexError: list index out of range

In [ ]:
df_coordinates = pd.DataFrame(all_coordinates)
df_combined = df.join(df_coordinates)
df_combined  = df_combined .rename(columns={0:'Latitude', 1:'Longitude'})

In [ ]:
df_combined.to_csv('resalehdb_data_withCoordinates.csv',index=False)